In [ ]:
import pandas as pd
import ipysheet

# Data set cleaning

## Load and overview of data set

In [ ]:
def load_data(file_name):
    data_df = pd.read_csv(file_name, sep=';')
    return data_df.dropna(axis=0, how='any', subset=['answer_normalised'])

In [ ]:
nlu_data_df = load_data(
    'NLU-Data-Home-Domain-Annotated-All.csv'
    )

In [ ]:
number_of_domains = len(nlu_data_df['scenario'].unique())
list_of_domains = nlu_data_df['scenario'].unique()

number_of_intents = nlu_data_df['intent'].nunique()
list_of_intents = nlu_data_df['intent'].unique()

number_of_utterances = nlu_data_df['answer_normalised'].nunique()

print(f'From a total of {number_of_utterances} utterances, there are {number_of_domains} domains, {number_of_intents} intents and {number_of_utterances} utterances.\n')

print(f'List of domains: {list_of_domains}\n')

print(f'List of intents: {list_of_intents}\n')

In [ ]:
from utils.nlu_engine import NLUEngine, LR

In [ ]:
domains = nlu_data_df.scenario.values
# TODO: remove vectorized_utterances from here and the function, it is not needed

LR_domain_classifier_model, tfidf_vectorizer = NLUEngine.train_intent_classifier(
    data_df_path=nlu_data_df,
    labels_to_predict='domain',
    classifier=LR
)


In [ ]:
utterance = "turn of the kitchen lights"

print(NLUEngine.predict_label(
    LR_domain_classifier_model, tfidf_vectorizer, utterance))


In [ ]:
domain_labels = 'scenario'

report_df = NLUEngine.evaluate_classifier(
    data_df_path=nlu_data_df,
    labels_to_predict=domain_labels,
    classifier=LR
)

In [ ]:
report_df

In [ ]:
from utils.nlu_engine import EntityExtractor

In [ ]:
utterance_with_tagging = 'wake me up at [time : five pm] [date : this week]'

entities = EntityExtractor.extract_entities(utterance_with_tagging)
entities


We don't want all of the columns, so we will drop some to review the data set

In [ ]:
nlu_scenario_data_df = nlu_data_df.drop(
    columns=[
        'userid', 'notes', 'answer', 'answer_normalised', 'answerid'
        ])


Pick a domain (scenario) to review

In [ ]:
nlu_scenario_df = nlu_scenario_data_df[
    nlu_scenario_data_df['scenario'] == 'iot'
    ]


## Convert to ipysheet and review

We shall make two buttons. 
* **review**: Either changes have been made or the entry should be further reviewed
* **remove**: We will drop the entry from the data set.

Look at each utterance, check the following:
* Is the utterance grammatically correct (and spelled correctly)?
* Is the utterance in the correct language?
* Is the utterance in the correct domain?
* Is the utterance in the correct format?
* Does the utterance actually make sense? (i.e. does it make sense to say it?)

If you are unsure, you are marking your changes as **review** anyway, so that's cool.

In [ ]:
nlu_scenario_df = nlu_scenario_df.assign(review=None)
nlu_scenario_df['review'] = nlu_scenario_df['review'].astype(bool)

nlu_scenario_df = nlu_scenario_df.assign(remove=None)
nlu_scenario_df['remove'] = nlu_scenario_df['remove'].astype(bool)

nlu_scenario_df_sheet = ipysheet.from_dataframe(nlu_scenario_df)
nlu_scenario_sheet = ipysheet.from_dataframe(nlu_scenario_df)
nlu_scenario_sheet

Once you are done reviewing, you convert it back to a dataframe and check to make sure it looks okay.

In [ ]:
reviwed_scenario_df = ipysheet.to_dataframe(nlu_scenario_sheet)
reviwed_scenario_df.index = pd.to_numeric(reviwed_scenario_df.index)
reviwed_scenario_df.tail(50)

If you are sure it is okay, you can save it as a csv file, make sure to name it correctly (i.e. `alarm_domain_first_review.csv`)

In [ ]:
reviwed_scenario_df.to_csv('audio_domain_first_review.csv')


Load it back up and check to make sure it looks okay. Make sure to give it the right name!

In [ ]:
audio_domain_first_review_df = pd.read_csv('audio_domain_first_review.csv', index_col=0)
audio_domain_first_review_df.tail(50)

In [ ]:
# TODO: implement the evaluate_classifier in the NLU engine to check f1 score for intents and entities in the domain vs original NLU data of domain!
# Value: benchmark!

In [ ]:
# TODO: cancat all reviewed dfs and save to csv

In [ ]:
# TODO: add benchmark for whole NLU data set before and after cleaning! (by intents and domains!)
# TODO: review the review marked entries
# TODO: add new column for notes
# TODO: change flow of review for only ones that should be reviewed, not all of the ones that have been changed (track changes by comparing against the original data set)
# TODO: do the changed utterances have to be changed in other fields too or is it just enough for the tagged utterancve field?

In [ ]:
# TODO: add visualizations of domains, their intents, keywords in utterances, and entities